# Koneoppiminen: Osa 6: Satunnaismetsä

In [ ]:
# Kirjoita tähän tietosi!
student_name = 'Tiina Jokela'

## Tehtävä 1
### Aihe: Tekstiilityöntekijöiden tuottavuus

4 pistettä

1. Tutustu ja lataa aineisto osoitteesta https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees# . Lataa aineisto DataFrameen. 

2. Jätä pois taustamuuttujat analysoitavasta datasta.

3. Ratkaise mitä teet puuttuville arvoille.

4. Normalisoi. 

5. Luo lisäksi luokkamuuttuja, joka on arvoltaa 0, jos tuottavuus on alle 0.5 ja arvoltaan 1, jos tuottavuus on yli 0.5. Tallenna tämä df-muuttujaan sarakkeeksi `class`.

In [2]:
# Kirjoita toteutuksesi tähän soluun. 
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import minmax_scale
 
df = pd.read_csv('garments_worker_productivity.csv') 
df = df.drop(['date', 'quarter', 'day', 'department', 'team'], axis=1)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(df).transform(df)

scaled = minmax_scale(imp, feature_range=(0, 1))
column_names = ('targeted_productivity', 'smv', 'wip', 'over_time', 'incentive', 'idle_time', 'idle_men', 'no_of_style_change', 
                'no_of_workers', 'actual_productivity')
df = pd.DataFrame(scaled, columns=column_names)
df.loc[df['actual_productivity'] < 0.5, 'class'] = 0
df.loc[df['actual_productivity'] >= 0.5, 'class'] = 1

In [3]:
# Tehtävän vastaukset. Huom! Älä muokkaa tätä solua, vaan aja se, kun olet suorittanut tehtävän. 
# Tallenna DataFrame df-muuttujaan ja koulutusaineisto.
print(f'Columns: {df.columns}')
print(f'First row by position: {df.iloc[0]}')

Columns: Index(['targeted_productivity', 'smv', 'wip', 'over_time', 'incentive',
       'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers',
       'actual_productivity', 'class'],
      dtype='object')
First row by position: targeted_productivity    1.000000
smv                      0.450252
wip                      0.047631
over_time                0.273148
incentive                0.027222
idle_time                0.000000
idle_men                 0.000000
no_of_style_change       0.000000
no_of_workers            0.655172
actual_productivity      0.797332
class                    1.000000
Name: 0, dtype: float64


## Tehtävä 2
### Aihe: Regressio

3 pistettä

1. Jaa aineisto koulutus- (67%) ja testausaineistoksi (33%) tallentaen myös luokkamuuttujat omiksi vektoreiksi (`X_train`, `y_train`, `X_test`, `y_test`). Käytä satunnaissiementä `23`.

Vinkki: älä laita sarakkeita `actual_productivity` tai `class` aineistoon `X`-muuttujiin.

2. Käytä `scikit-learn`-kirjaston `ensemble`-moduulin [funktioita](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) ja luo regressiomalli satunnaismetsää hyödyntäen. Yritetään siis ennustaa `actual_productivity`-muuttujaa. 

Käytä satunnaissiementä `753`.

3. Ennusta testausaineiston `actual_productivity`. Tallenna selitetty varianssi `exp_var` ja suurin virhe `max_err` kirjaston `metrics`-moduulin työkaluilla.

In [5]:
# Kirjoita toteutuksesi tähän soluun. 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error

X = df.drop(['actual_productivity', 'class'], axis=1)
y = df['actual_productivity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=23)

regr = RandomForestRegressor(random_state=753)
y_pred = regr.fit(X_train, y_train).predict(X_test)

exp_var = explained_variance_score(y_test, y_pred)
max_err = max_error(y_test, y_pred)

In [6]:
# Tehtävän vastaukset. Huom! Älä muokkaa tätä solua, vaan aja se, kun olet suorittanut tehtävän. 
# Tallenna vastaukset exp_var- ja max_err-muuttujiin.
print(f'Test length: {len(X_test)}')
print(f'Train length: {len(X_train)}')
print(f'Train columns: {X_train.columns}')
print(f'Explained variance: {exp_var}')
print(f'Maximum error: {max_err}')

Test length: 396
Train length: 801
Train columns: Index(['targeted_productivity', 'smv', 'wip', 'over_time', 'incentive',
       'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers'],
      dtype='object')
Explained variance: 0.36585100868829734
Maximum error: 0.6626927941429283


## Tehtävä 3
### Aihe: Luokittelu

3 pistettä

1. Muodosta koulutus- ja testausaineisto. Käytä satunnaissiementä `23`. Huomioi, että nyt ennustetaan `class`-muuttujaa.

2. Käytä `scikit-learn`-kirjaston `ensemble`-moduulin [funktioita](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) ja luo luokittelumalli satunnaismetsää hyödyntäen. Käytä satunnaissiementä 753. Yritetään siis ennustaa luokkamuuttujaa `class`. 

3. Ennusta testausaineistosta `class`. Luo tarkkuusraportti `cr` ja sekaannusmatriisi `cm` kirjaston `metrics`-moduulin työkaluilla.

In [9]:
# Kirjoita toteutuksesi tähän soluun. 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=23)

clf = RandomForestClassifier(random_state=753)
y_pred = clf.fit(X_train, y_train).predict(X_test)

cr = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

In [10]:
# Tehtävän vastaukset. Huom! Älä muokkaa tätä solua, vaan aja se, kun olet suorittanut tehtävän. 
# Tulokset ovat cm- ja cr-muuttujissa. 
print(f'Test length: {len(X_test)}')
print(f'Train length: {len(X_train)}')
print(cr)
print(cm)

Test length: 396
Train length: 801
              precision    recall  f1-score   support

         0.0       0.73      0.62      0.67       107
         1.0       0.87      0.92      0.89       289

    accuracy                           0.84       396
   macro avg       0.80      0.77      0.78       396
weighted avg       0.83      0.84      0.83       396

[[ 66  41]
 [ 24 265]]
